In [1]:
from groq import Groq
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

In [2]:
client = Groq(api_key=groq_api_key)

# Define the model
model_name = "mixtral-8x7b-32768"

In [3]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [5]:
def call_groq(messages):
    """Call Groq API and return response."""
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0,
        max_tokens=1000
    )
    return response.choices[0].message.content

In [6]:
def groq_chain(input_data):
    """Custom function to integrate Groq API with LangChain."""
    user_input = input_data["input"]
    session_id = input_data["session_id"]
    history = get_chat_history(session_id)

    # Construct message history for API call
    messages = [{"role": "system", "content": "You are a helpful AI assistant."}]
    messages += [{"role": msg.type, "content": msg.content} for msg in history.messages]
    messages.append({"role": "user", "content": user_input})

    # Get AI response
    assistant_reply = call_groq(messages)

    # Update chat history
    history.add_user_message(user_input)
    history.add_ai_message(assistant_reply)

    return assistant_reply

In [7]:
from langchain_core.runnables import RunnableLambda
groq_runnable = RunnableLambda(groq_chain)


chain_with_history = RunnableWithMessageHistory(
    groq_runnable,  
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [8]:
session_id = "user_123"

response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?", "session_id": session_id},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1)

AI: Hello! I'm an AI and I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [9]:
response2 = chain_with_history.invoke(
    {"input": "What was my previous message?", "session_id": session_id},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2)

BadRequestError: Error code: 400 - {'error': {'message': "'messages.1' : discriminator property 'role' has invalid value", 'type': 'invalid_request_error'}}

In [10]:

# Print conversation history
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you?
ai: Hello! I'm an AI and I don't have feelings, but I'm here and ready to help you. How can I assist you today?
human: Hello! How are you?
ai: Hello! I'm an AI and I don't have feelings, but I'm here and ready to help you. How can I assist you today?
